In [5]:
import os
from langchain_openai import ChatOpenAI
import textwrap

os.environ["OPENAI_API_KEY"] = "none"

model = ChatOpenAI(
    model="qwen3:1.7b",
    base_url="http://localhost:11434/v1",
    )

response = model.stream("tell me a joke.")
# response.content

for chunk in response:
    print(chunk.content, end="", flush=True)

Sure! Here's a light-hearted joke:  

**"Why don't skeletons fight each other? They don't have the guts!"**  

😄 Let me know if you want another one!

In [21]:
import os
from langchain_openai import ChatOpenAI
import textwrap

os.environ["OPENAI_API_KEY"] = "none"

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

model = ChatOpenAI(
    model="functiongemma:270m",
    base_url="http://localhost:11434/v1",
    tools=[get_weather]
    )

# response = model.invoke("Why do parrots talk?")
response = model.invoke({"messages": [{"role": "user", "content": "what is the weather in sf"}]})
response.content

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.

In [23]:
import os
from langchain_openai import ChatOpenAI

# 1. Setup environment
os.environ["OPENAI_API_KEY"] = "ollama" # Can be anything for local

def get_weather(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

# 2. Initialize the model
model = ChatOpenAI(
    model="functiongemma:270m",
    base_url="http://localhost:11434/v1",
    temperature=0
)

# 3. Bind the tools to the model
# This tells the model what functions it is allowed to "call"
model_with_tools = model.bind_tools([get_weather])

# 4. Invoke the model
response = model_with_tools.invoke("what is the weather in sf")

# 5. Inspect the tool calls
print(f"Content: {response.content}")
print(f"Tool Calls: {response.tool_calls}")


Content: 
Tool Calls: [{'name': 'get_weather', 'args': {'city': 'sf'}, 'id': 'call_v74wjrqd', 'type': 'tool_call'}]


In [24]:
from dataclasses import dataclass
from langchain_openai import ChatOpenAI

from langchain.agents import create_agent
from langchain.chat_models import init_chat_model
from langchain.tools import tool, ToolRuntime
from langgraph.checkpoint.memory import InMemorySaver
from langchain.agents.structured_output import ToolStrategy


# Define system prompt
SYSTEM_PROMPT = """You are an expert weather forecaster, who speaks in puns.

You have access to two tools:

- get_weather_for_location: use this to get the weather for a specific location
- get_user_location: use this to get the user's location

If a user asks you for the weather, make sure you know the location. If you can tell from the question that they mean wherever they are, use the get_user_location tool to find their location."""

# Define context schema
@dataclass
class Context:
    """Custom runtime context schema."""
    user_id: str

# Define tools
@tool
def get_weather_for_location(city: str) -> str:
    """Get weather for a given city."""
    return f"It's always sunny in {city}!"

@tool
def get_user_location(runtime: ToolRuntime[Context]) -> str:
    """Retrieve user information based on user ID."""
    user_id = runtime.context.user_id
    return "Florida" if user_id == "1" else "SF"

# Define response format
@dataclass
class ResponseFormat:
    """Response schema for the agent."""
    # A punny response (always required)
    punny_response: str
    # Any interesting information about the weather if available
    weather_conditions: str | None = None

# Set up memory
checkpointer = InMemorySaver()

# Create agent
os.environ["OPENAI_API_KEY"] = "ollama"
agent = ChatOpenAI(
    model="functiongemma:270m",
    base_url="http://localhost:11434/v1",
    system_prompt=SYSTEM_PROMPT,
    tools=[get_user_location, get_weather_for_location],
    context_schema=Context,
    response_format=ToolStrategy(ResponseFormat),
    checkpointer=checkpointer
)

# Run agent
# `thread_id` is a unique identifier for a given conversation.
config = {"configurable": {"thread_id": "1"}}

response = agent.invoke(
    {"messages": [{"role": "user", "content": "what is the weather outside?"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="Florida is still having a 'sun-derful' day! The sunshine is playing 'ray-dio' hits all day long! I'd say it's the perfect weather for some 'solar-bration'! If you were hoping for rain, I'm afraid that idea is all 'washed up' - the forecast remains 'clear-ly' brilliant!",
#     weather_conditions="It's always sunny in Florida!"
# )


# Note that we can continue the conversation using the same `thread_id`.
response = agent.invoke(
    {"messages": [{"role": "user", "content": "thank you!"}]},
    config=config,
    context=Context(user_id="1")
)

print(response['structured_response'])
# ResponseFormat(
#     punny_response="You're 'thund-erfully' welcome! It's always a 'breeze' to help you stay 'current' with the weather. I'm just 'cloud'-ing around waiting to 'shower' you with more forecasts whenever you need them. Have a 'sun-sational' day in the Florida sunshine!",
#     weather_conditions=None
# )

c:\Users\pkmpp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! system_prompt is not default parameter.
                system_prompt was transferred to model_kwargs.
                Please confirm that system_prompt is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\pkmpp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! tools is not default parameter.
                tools was transferred to model_kwargs.
                Please confirm that tools is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\pkmpp\anaconda3\Lib\site-packages\IPython\core\interactiveshell.py:3639: UserWarning: WARNING! context_schema is not default parameter.
                context_schema was transferred to model_kwargs.
                Please confirm that context_schema is what you intended.
  if await self.run_code(code, result, async_=asy):
c:\Users\pkmpp\anaconda3\Lib

ValueError: Invalid input type <class 'dict'>. Must be a PromptValue, str, or list of BaseMessages.